In [10]:
# Imports
import librosa, librosa.display
import matplotlib.pyplot as plt
import os
import numpy as np
import math
import json
import cv2
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import pickle
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Embedding
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch
from keras_tuner.tuners import Hyperband
from tensorflow import keras

In [11]:
# Load data
DATA_PATH = "JSON/TrainData.json"

SAMPLE_RATE = 22050
DURATION = 4
SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION

def load_audio_data(data_path):
    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    return X, y

In [12]:
X_audio, y_audio = load_audio_data(DATA_PATH)
X_audio = X_audio[..., np.newaxis]

In [13]:
class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        
        model = Sequential()
        
        # Convolutional 1
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'Cov_filters_1',
                    values=[16, 32, 64],
                    default=32,
                ),
                activation='relu',
                kernel_size=(3,3),
                input_shape=(X_audio.shape[1], X_audio.shape[2], X_audio.shape[3])
            )
        )
        
        # Convolutional 2
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'Cov_filters_2',
                    values=[16, 32, 64],
                    default=32,
                ),
                activation='relu',
                kernel_size=(3,3)
            )
        )

        # Pooling 1
        model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
        
        # Dropout 1
        model.add(
            Dropout(rate=hp.Float(
                'dropout_1',
                min_value=0.0,
                max_value=0.5,
                default=0.2,
                step=0.05
            ))
        )
        
        # Convolutional 3
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'Cov_filters_3',
                    values=[16, 32, 64],
                    default=64,
                ),
                activation='relu',
                kernel_size=(3,3)
            )
        )
        
        # Convolutional 4
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'Cov_filters_4',
                    values=[16, 32, 64],
                    default=64,
                ),
                activation='relu',
                kernel_size=(3,3)
            )
        )

        # Pooling 2
        model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
        
        # Dropout 2
        model.add(
            Dropout(rate=hp.Float(
                'dropout_2',
                min_value=0.0,
                max_value=0.5,
                default=0.1,
                step=0.05
            ))
        )
        
        # Flatten
        model.add(Flatten())
        
        # Hidden layer 1
        model.add(
            Dense(
                units=hp.Int(
                    'units_1',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=184
                ),
                activation='relu'
            )
        )
        
        # Dropout 1
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_1',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.05,
                    step=0.05
                )
            )
        )
        
        # Hidden layer 2
        model.add(
            Dense(
                units=hp.Int(
                    'units_2',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=184
                ),
                activation='relu'
            )
        )
        
        # Dropout 2
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_2',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.05,
                    step=0.05
                )
            )
        )
        
        # Hidden layer 3
        model.add(
            Dense(
                units=hp.Int(
                    'units_3',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=184
                ),
                activation='relu'
            )
        )
        
        # Dropout 3
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_3',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.05,
                    step=0.05
                )
            )
        )
        
        # Hidden layer 4
        model.add(
            Dense(
                units=hp.Int(
                    'units_4',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=62
                ),
                activation='relu'
            )
        )
        
        # Dropout 4
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_4',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.05,
                    step=0.05
                )
            )
        )
        
        # Hidden layer 5
        model.add(
            Dense(
                units=hp.Int(
                    'units_5',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=184
                ),
                activation='relu'
            )
        )
        
        # Dropout 4
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_5',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05
                )
            )
        )
        
        # Hidden layer 6
        model.add(
            Dense(
                units=hp.Int(
                    'units_6',
                    min_value=32,
                    max_value=512,
                    step=32,
                    default=256
                ),
                activation='relu'
            )
        )
        
        # Dropout 6
        model.add(
            Dropout(
                rate=hp.Float(
                    'dropout_6',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.14,
                    step=0.05
                )
            )
        )

        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling='LOG',
                    default=1e-3
            )),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        return model
    
NUM_CLASSES = 3  # happy neutral sad number of classes
INPUT_SHAPE = (X_audio.shape[1], X_audio.shape[2], X_audio.shape[3])
SEED = 1
HYPERBAND_MAX_EPOCHS = 15
MAX_TRIALS = 25
EXECUTION_PER_TRIAL = 2

hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    seed=SEED,
    max_trials=MAX_TRIALS,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='random_search',
    project_name='CNNAF2'
)

tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_accuracy',
    seed=SEED,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='hyperband',
    project_name='CNNAF2'
)

N_EPOCH_SEARCH = 15
BATCH_SIZE = 4

tuner.search(X_audio, y_audio, batch_size=BATCH_SIZE, epochs=N_EPOCH_SEARCH, validation_split=0.2)

tuner.results_summary()

# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]

Trial 30 Complete [00h 00m 52s]
val_accuracy: 0.5

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 12m 51s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in hyperband\CNNAF2
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
Cov_filters_1: 16
Cov_filters_2: 16
dropout_1: 0.1
Cov_filters_3: 16
Cov_filters_4: 64
dropout_2: 0.4
units_1: 192
units_2: 128
units_3: 256
dropout_3: 0.30000000000000004
units_4: 256
dropout_4: 0.25
units_5: 160
dropout_5: 0.05
units_6: 480
dropout_6: 0.2
learning_rate: 0.00110654178783952
tuner/epochs: 15
tuner/initial_epoch: 5
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 2e23bcd4621ba9c76e57fa96b98a3b38
Score: 1.0
Trial summary
Hyperparameters:
Cov_filters_1: 32
Cov_filters_2: 16
dropout_1: 0.25
Cov_filters_3: 32
Cov_filters_4: 64
dropout_2: 0.2
units_1: 288
units_2: 224
units_3: 96
dropout_3: 0.25
units_4: 64
dropout_4: 0.45
units_5: 480
dropout_5: 0.15000000000000002
units_6: 352
dropo